In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling

import lightgbm as lgb
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from category_encoders import target_encoder

import gc
from pycaret.regression import *

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder

In [2]:
# df_train_path = '../input/jeju_traffic/train.csv'
# df_test_path = '../input/jeju_traffic/test.csv'

# def csv_to_parquet(csv_path, save_name):
#     df = pd.read_csv(csv_path)
#     df.to_parquet(f'./{save_name}.parquet')
#     del df
#     gc.collect()
#     print(save_name, 'Done.')

# csv_to_parquet(df_train_path, '../input/jeju_traffic/train')
# csv_to_parquet(df_test_path, '../input/jeju_traffic/test')

In [2]:
df_info_path = '../input/jeju_traffic/data_info.csv'
df_train_path = '../input/jeju_traffic/train.parquet'
df_test_path = '../input/jeju_traffic/test.parquet'

df_info = pd.read_csv(df_info_path)
df_train = pd.read_parquet(df_train_path)
df_test = pd.read_parquet(df_test_path)

# df_info

In [77]:
df_train.profile_report()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [5]:
# df_test.profile_report()

In [67]:
df_train_proc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4701217 entries, 0 to 4701216
Data columns (total 14 columns):
 #   Column                 Dtype   
---  ------                 -----   
 0   day_of_week            category
 1   base_hour              category
 2   lane_count             category
 3   road_rating            category
 4   multi_linked           category
 5   connect_code           category
 6   maximum_speed_limit    category
 7   weight_restricted      category
 8   road_type              category
 9   start_turn_restricted  category
 10  end_turn_restricted    category
 11  target                 float64 
 12  start_pos              category
 13  end_pos                category
dtypes: category(13), float64(1)
memory usage: 103.2 MB


In [7]:
(100*df_train[:10]['start_longitude'] + df_train[:10]['start_latitude']).astype(str)

0    12699.688951139884
1    12686.411406584704
2      12670.1389111909
3    12689.966511899634
4    12666.117325468114
5    12683.816317219576
6    12660.221314509074
7    12677.644564795575
8    12683.892347803023
9    12710.057502360429
dtype: object

In [8]:
# # check uniqueness of start pos
# df_train['start'] = 100.*df_train['start_longitude'] + df_train['start_latitude']
# df_train['start'].drop_duplicates()

0        12699.688951
1        12686.411407
2        12670.138911
3        12689.966512
4        12666.117325
             ...     
4589     12726.554373
4764     12668.351005
6964     12666.676779
10399    12698.828345
11784    12701.718455
Name: start, Length: 586, dtype: float64

In [9]:
# # check uniqueness of end pos
# df_train['end'] = 100.*df_train['end_longitude'] + df_train['end_latitude']
# df_train['end'].drop_duplicates()

0        12699.661248
1        12686.128819
2        12669.494822
3        12689.868389
4        12666.477856
             ...     
4633     12710.023496
5920     12660.138130
5928     12686.448885
6964     12666.789875
11784    12701.568198
Name: end, Length: 586, dtype: float64

In [13]:
def df_preprocessing(df, flag_target=True, te=None):
    not_used_columns = ['id','base_date', 'road_name', 'start_node_name', 'end_node_name', 'vehicle_restricted', 'height_restricted']
    for not_used_column in not_used_columns:
        if not_used_column in df.columns:
            df = df.drop(not_used_column, axis=1)
    
    df['start_pos'] = (100.*df['start_longitude'] + df['start_latitude']).astype(str)
    df['end_pos'] = (100.*df['end_longitude'] + df['end_latitude']).astype(str)

    cat_cols = ['day_of_week','start_turn_restricted','end_turn_restricted', 'start_pos', 'end_pos', \
        'base_hour', 'lane_count', 'road_rating', 'multi_linked', 'connect_code', 'maximum_speed_limit', 'weight_restricted', 'road_type']

    df_c = df[cat_cols]
    if flag_target:
        te = target_encoder.TargetEncoder()    
        df_c = te.fit_transform(df_c, df['target'])
    else:
        df_c = te.transform(df_c)

    df_c = pd.DataFrame(df_c, index = df.index, columns=cat_cols)
    for i, column in enumerate(df_c.columns):
        df[column] = df_c[column]

    df = df.drop(['start_longitude', 'start_latitude', 'end_longitude', 'end_latitude'], axis=1)
    if flag_target:
        return df, te
    else: return df

In [14]:
target = df_train['target']
df_train_proc, te = df_preprocessing(df_train, flag_target=True)
df_test_proc = df_preprocessing(df_test, flag_target=False, te=te)

In [7]:
df_train_proc

,day_of_week,base_hour,lane_count,road_rating,multi_linked,connect_code,maximum_speed_limit,weight_restricted,road_type,start_turn_restricted,end_turn_restricted,target,start_pos,end_pos
0,42.768345,17,1,106,0,0,60.0,32400.0,3,43.610246,43.64559,52.0,49.521125,51.756910
1,42.768345,21,2,103,0,0,60.0,0.0,0,35.238967,43.64559,30.0,26.400712,26.400712
2,43.179300,7,2,103,0,0,80.0,0.0,0,43.610246,43.64559,61.0,59.101720,65.411434
3,42.450328,13,2,107,0,0,50.0,0.0,0,43.610246,43.64559,20.0,23.755158,25.445418
4,42.699198,8,2,103,0,0,80.0,0.0,0,43.610246,43.64559,38.0,39.873670,39.873670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4701212,42.768345,16,1,107,0,0,50.0,0.0,0,43.610246,43.64559,20.0,27.620482,28.384300
4701213,42.768345,2,2,107,0,0,80.0,43200.0,3,43.610246,43.64559,65.0,65.385516,68.900580
4701214,42.761365,22,2,103,0,0,60.0,0.0,0,43.610246,43.64559,30.0,31.988019,41.850580
4701215,42.770372,2,2,103,0,0,80.0,0.0,0,43.610246,43.64559,73.0,61.554966,59.264147


In [8]:
X_train = df_train_proc.drop('target', axis=1)
y_train = df_train_proc['target']
LR = lgb.LGBMRegressor(random_state=42).fit(X_train, y_train)


In [15]:
pred = LR.predict(df_test_proc)
pred

array([24.96164368, 41.73918777, 61.70206594, ..., 25.37111799,
       24.90603663, 45.16647036])

In [16]:
sample_submission = pd.read_csv('../input/jeju_traffic/sample_submission.csv')
sample_submission['target'] = pred
sample_submission.to_csv('../input/jeju_traffic/submit.csv', index = False)
sample_submission

,id,target
0,TEST_000000,24.961644
1,TEST_000001,41.739188
2,TEST_000002,61.702066
3,TEST_000003,37.668970
4,TEST_000004,41.200472
...,...,...
291236,TEST_291236,46.905753
291237,TEST_291237,53.179050
291238,TEST_291238,25.371118
291239,TEST_291239,24.906037


In [15]:
reg = setup(data=df_train_proc, 
            target='target',
            categorical_features=['base_hour', 'weight_restricted'], # 범주형 컬럼 지정
            # numeric_features=numeric_features,         # 수치형 컬럼 지정
            normalize=True,                            # 정규화 적용
            normalize_method='zscore',                 # 정규화 방식 지정
            imputation_type='iterative',               # 결측치를 lightgbm으로 예측하여 채움
            iterative_imputation_iters=10,             # imputation iteration 지정
            categorical_iterative_imputer='lightgbm',
            polynomial_features=True,
            session_id=42, 
            use_gpu=True,
            # silent=True,
            ) 

IntProgress(value=0, description='Processing: ', max=3)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,16:45:31
Status,. . . . . . . . . . . . . . . . . .,Preprocessing Data


Text(value="Following data types have been inferred automatically, if they are correct press enter to continue…

,Data Type
day_of_week,Categorical
base_hour,Categorical
lane_count,Categorical
road_rating,Categorical
multi_linked,Categorical
connect_code,Categorical
maximum_speed_limit,Numeric
weight_restricted,Categorical
road_type,Categorical
start_turn_restricted,Categorical
